In [354]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [355]:
#Test Code| Version 2#
#Read Motherdata
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

In [356]:
#Test code| Version 2#
Year='1934'
Showa='9'
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

PositionList=['Manager','Leader','Admin','Outsource','Engineer1','Engineer2']

In [357]:
#Test code| Version 2#
#Show Working office#
n=2

#Extract key info of office
Row  = df.iloc[n]
NxRow= df.iloc[n+1]
Row, NxRow

(Office      文書課
 Dept      Admin
 Year          9
 Page          3
 Name: 2, dtype: object,
 Office    庶務課
 Dept      監査局
 Year        9
 Page        5
 Name: 3, dtype: object)

In [358]:
# Function for returning x coordinate of Position name
def Detect_Position(Json,Position):
    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue
            
    ### Get Coordinates ###
    if Position=='Manager':        
        res = [d
           for d in Dict 
           if ('主事' in d['inferText']) or ('主' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Managers on this page!")
            return(Edge)
        else:
            print("No Managers on this page!")
            return([])
    
    if Position=='Leader':        
        res = [d
           for d in Dict 
           if ('課長' in d['inferText']) or ('課' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Leader on this page!")
            return(Edge)
        else:
            print("No Leader on this page!")
            return([])
        
    if Position=='Engineer1':        
        res = [d
           for d in Dict 
           if ('技師' in d['inferText']) or ('師' in d['inferText'])]
        print(res)
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Engineer1 on this page!")
            return(Edge)
        else:
            print("No Engineer1 on this page!")
            return([])
    
    if Position=='Engineer2':        
        res = [d
           for d in Dict 
           if ('技手' in d['inferText']) or ('手' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Engineer2 on this page!")
            return(Edge)
        else:
            print("No Engineer2 on this page!")
            return([])
        
    if Position=='Admin':        
        res = [d
           for d in Dict 
           if ('事務員' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Admin on this page!")
            return(Edge)
        else:
            print("No Admin on this page!")
            return([])
        
    if Position=='Outsource':
        res = [d
           for d in Dict 
           if '嘱託員' in d['inferText']]
        print(res)
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Outsource on this page!")
            return(Edge)
        else:
            print("No Engineer on this page!")
            return([])
        
    if Position=='Clerk':
        res = [d
           for d in Dict 
           if d['inferText']=='書記']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            print("No Clerk on this page!")
            return([])
        
    if Position=='Worker':
        res = [d
           for d in Dict 
           if d['inferText']=='雇']
        print(res)
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            print("No Worker on this page!")
            return([])
        
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Dept,Office,ImagePage):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"
    with open(path+"Image"+str(ImagePage)+".png",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [359]:
###Collect Location information###
Page=int(Row["Page"])
NxPage=int(NxRow["Page"])

Dept=Row["Dept"]
NxDept=NxRow["Dept"]

Office=Row["Office"]
NxOffice=NxRow["Office"]

#Convert to json via CLOVA
JsonList=[]
for n in range(1,len(dta[str(Year)][Dept][Office]['Page_Range'])+1):
    Json=Clova(Dept,Office,n)
    JsonList.append(Json)

In [360]:
#Find X coordinate of Positions.
PositionListNow=PositionList.copy()
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
Page_Range=dta[str(Year)][Dept][Office]['Page_Range']
CordDict={}
Page=Page_Range[0]
for k in range(0,len(JsonList)):
    print('Looking at Image'+str(k+1))
    
    CordList=list(map(Detect_Position,[JsonList[k]]*len(PositionListNow),PositionListNow))
    Positions= {PositionListNow[i]:CordList[i] for i in range(len(PositionListNow))}
    Positions = {k:v for k,v in Positions.items() if v}    
    
    PosiList = list(Positions.keys())
    CordList = list(Positions.values())
    
    CordDictNow= {PosiList[i]: {'XLocation':CordList[i],'Page':Page} for i in range(len(PosiList))}
    CordDictNow = {k:v for k,v in CordDictNow.items() if v}
    CordDict.update(CordDictNow)
    Page=Page+1

    for x in list(CordDictNow.keys()):
      PositionListNow.remove(x)
    
    print(PositionListNow)
    if len(PositionListNow)==0:
        break
dta[Year][Dept][Office]['Position']=CordDict

Looking at Image1
Found Managers on this page!
Found Leader on this page!
No Admin on this page!
[]
No Engineer on this page!
[]
No Engineer1 on this page!
No Engineer2 on this page!
['Admin', 'Outsource', 'Engineer1', 'Engineer2']
Looking at Image2
Found Admin on this page!
[{'inferText': '嘱託員', 'boundingPoly': {'vertices': [{'x': 286.0, 'y': 74.0}, {'x': 302.0, 'y': 74.0}, {'x': 302.0, 'y': 116.0}, {'x': 286.0, 'y': 116.0}]}}]
Found Outsource on this page!
[]
No Engineer1 on this page!
No Engineer2 on this page!
['Engineer1', 'Engineer2']
Looking at Image3
[]
No Engineer1 on this page!
No Engineer2 on this page!
['Engineer1', 'Engineer2']


In [361]:
##Add Ending information
Frame=pd.DataFrame.from_dict(dta[Year][Dept][Office]['Position'],orient='index')
Frame["EndLocation"]=list(Frame['XLocation'].shift(-1))[:-1]+list([dta[Year][Dept][Office]["Office_X2"]])
Frame["EndPage"]=list(Frame['Page'].shift(-1))[:-1]+list([dta[Year][NxDept][NxOffice]["Starting_Page"]])
Frame=Frame.to_dict('index')
dta[Year][Dept][Office]['Position']=Frame

In [363]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)